In [1]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

In [2]:
# !unzip data.zip

In [3]:
import numpy as np
import torch

In [4]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
torch.__version__

'2.9.1+cpu'

In [6]:
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class HairClassifier(nn.Module):
    def __init__(self, num_classes=1, input_image_size=(200, 200)):
        super(HairClassifier, self).__init__()

        self.conv = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Dynamically calculate input features for fc1
        # Create a dummy tensor to pass through the conv and pool layers
        dummy_input = torch.zeros(1, 3, input_image_size[0], input_image_size[1])
        x = self.pool(F.relu(self.conv(dummy_input)))
        fc1_input_features = torch.flatten(x, 1).shape[1]

        self.fc1 = nn.Linear(fc1_input_features, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Question 2

In [7]:
model = HairClassifier(num_classes=1)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
import os

from PIL import Image
from torch.utils.data import Dataset

class HairDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data_dir = data_dir
    self.transform = transform
    self.image_paths = []
    self.labels = []
    self.classes = sorted(os.listdir(data_dir))
    self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

    for label_name in self.classes:
      label_dir = os.path.join(data_dir, label_name)
      for img_name in os.listdir(label_dir):
        self.image_paths.append(os.path.join(label_dir, img_name))
        self.labels.append(self.class_to_idx[label_name])

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
     img_path = self.image_paths[idx]
     image = Image.open(img_path).convert("RGB")
     label = self.labels[idx]

     if self.transform:
      image = self.transform(image)

     return image, label


In [ ]:
from torchvision import transforms

input_size = 200
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std) # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std) # ImageNet normalization
])

In [ ]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=test_transforms
)

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def train_and_evaluate(train_loader, test_loader, num_epochs=10):
  history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      correct_train = 0
      total_train = 0
      for images, labels in train_loader:
          images, labels = images.to(device), labels.to(device)
          labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * images.size(0)
          # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
          predicted = (torch.sigmoid(outputs) > 0.5).float()
          total_train += labels.size(0)
          correct_train += (predicted == labels).sum().item()

      epoch_loss = running_loss / len(train_dataset)
      epoch_acc = correct_train / total_train
      history['loss'].append(epoch_loss)
      history['acc'].append(epoch_acc)

      model.eval()
      val_running_loss = 0.0
      correct_val = 0
      total_val = 0
      with torch.no_grad():
          for images, labels in test_loader:
              images, labels = images.to(device), labels.to(device)
              labels = labels.float().unsqueeze(1)

              outputs = model(images)
              loss = criterion(outputs, labels)

              val_running_loss += loss.item() * images.size(0)
              predicted = (torch.sigmoid(outputs) > 0.5).float()
              total_val += labels.size(0)
              correct_val += (predicted == labels).sum().item()

      val_epoch_loss = val_running_loss / len(test_dataset)
      val_epoch_acc = correct_val / total_val
      history['val_loss'].append(val_epoch_loss)
      history['val_acc'].append(val_epoch_acc)

      print(f"Epoch {epoch+1}/{num_epochs}, "
            f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
            f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

  return history

In [ ]:
history = train_and_evaluate(train_loader, test_loader, num_epochs=10)

### Question 3

In [ ]:
med_acc = np.median(history['acc'])
print(f"The median of training accuracy over all epochs is {med_acc:.2f}.")

### Question 4

In [ ]:
std_loss = np.std(history['loss'])
print(f"The standard deviation of training losses is {std_loss:.3f}.")

### Question 5

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=mean,std=std) # ImageNet normalization
])

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
history = train_and_evaluate(train_loader, test_loader, num_epochs=10)

In [ ]:
mean_test_loss = np.mean(history['val_loss'])
print(f"The mean of test loss for all the epochs for the model trained with augmentations is {mean_test_loss:.2f}.")

### Question 6

In [ ]:
ave_test_acc = np.mean(history['val_acc'][-5:])
print(f"The average test accuracy for the last 5 epochs for the model trained with augmentations is {ave_test_acc:.2f}.")